In [1]:
# https://k3no.medium.com/how-to-query-in-graphql-6ebb3f7085dc
# https://github.com/sushiswap/sushiswap-subgraph

In [2]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd
import dataframe_image as dfi
import os
from datetime import datetime
import numpy as np
pd.set_option('display.max_rows', 99999)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
def client(url):
    sample_transport = RequestsHTTPTransport(url = url,
                                             verify = True,
                                             retries = 3)
    
    client = Client(transport=sample_transport)
    return client

In [4]:
# interesting tokens on MATIC

 #https://polygon.fulcrum.trade/borrow/asset
lending_tokens_fulcrum = ['LINK', 'BZRX', 'AAVE', 'WBTC', 'USDC', 'USDT', 'WETH', 'WMATIC']

# https://creamfinance.substack.com/p/cream-communications-14
lending_tokens_aave = ['MATIC', 'DAI', 'USDC', 'USDT', 'WETH', 'WBTC', 'AAVE']
lending_tokens_cream = ['USDC', 'USDT', 'DAI', 'WMATIC', 'WETH', 'WBTC', 'LINK', 'SUSHI', 'CRV', 'QUICK']
remaining_interesting_tokens = ['FRAX', 'ibBTC']

interesting_tokens = list(set(lending_tokens_fulcrum +\
                              lending_tokens_aave +\
                              lending_tokens_cream +\
                              remaining_interesting_tokens))

interesting_tokens

['WETH',
 'USDC',
 'DAI',
 'LINK',
 'CRV',
 'AAVE',
 'MATIC',
 'SUSHI',
 'WMATIC',
 'ibBTC',
 'USDT',
 'WBTC',
 'BZRX',
 'QUICK',
 'FRAX']

## Quickswap

In [5]:
query = gql('''
query {
  uniswapFactories {
    pairCount
  }
}
''')

response = client('https://api.thegraph.com/subgraphs/name/sameepsi/quickswap02').execute(query)
number_of_pairs = int(response['uniswapFactories'][0]['pairCount'])
print(number_of_pairs)

6151


In [6]:
try:
    df = pd.read_excel('pairdata_Quickswap_MATIC.xlsx')
except:
    df = pd.DataFrame([])
    response_dict = {}
    response_list = []


    for i in range(number_of_pairs):
        if i % 100 == 0:
            querystring = '''
                  query {
                  pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                          token0 {symbol}
                          token1 {symbol}
                          id
                      }
                    }
                    '''
            try:
                response = client.execute(gql(querystring))
                df = df.append(pd.DataFrame.from_dict(response['pairs']))
                print(len(df))
            except:
                pass

In [7]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    df.head(2)

In [8]:
criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.to_excel('pairdata_Quickswap_MATIC.xlsx', index = False)
df_filtered.head()

,id,Pair,Token 0,Token 1
0,0x019ba0325f1988213d448b3472fa1cf8d07618d7,WMATIC-QUICK,WMATIC,QUICK
1,0x02e0e14d65175420fb12acd325a7be456bc22005,QUICK-DAI,QUICK,DAI
2,0x0554059d42e26f35cc958581c71fdfd92405d02f,WMATIC-AAVE,WMATIC,AAVE
3,0x0b3a76d33d93b2651dbf03b102eefa9500e89243,SUSHI-WMATIC,SUSHI,WMATIC
4,0x1bd06b96dd42ada85fdd0795f3b4a79db914add5,WETH-QUICK,WETH,QUICK


In [9]:
df4 = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client('https://api.thegraph.com/subgraphs/name/moonster-bsc/quickswap-daypairdataslim')\
                      .execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df4 = df4.append(df1)
    
df4 = df4[['id', 'timestamp', 'volumeUSD']]

 1 / 47 pair: 0x019ba0325f1988213d448b3472fa1cf8d07618d7
 2 / 47 pair: 0x02e0e14d65175420fb12acd325a7be456bc22005
 3 / 47 pair: 0x0554059d42e26f35cc958581c71fdfd92405d02f
 4 / 47 pair: 0x0b3a76d33d93b2651dbf03b102eefa9500e89243
 5 / 47 pair: 0x1bd06b96dd42ada85fdd0795f3b4a79db914add5
 6 / 47 pair: 0x1f1e4c845183ef6d50e9609f16f6f9cae43bc9cb
 7 / 47 pair: 0x2aa7a18ceabf2ef893d2f7c0145cc45e6f10b223
 8 / 47 pair: 0x2cf7252e74036d1da831d11089d326296e64a728
 9 / 47 pair: 0x3c986748414a812e455dcd5418246b8fded5c369
10 / 47 pair: 0x4a35582a710e1f4b2030a3f826da20bfb6703c09
11 / 47 pair: 0x4f2cc25028d4d80355c802482463b5b2a93cdb84
12 / 47 pair: 0x527a26525f5483612a1f96cfceacc7032b1c8610
13 / 47 pair: 0x59153f27eefe07e5ece4f9304ebba1da6f53ca88
14 / 47 pair: 0x5ca6ca6c3709e1e6cfe74a50cf6b2b6ba2dadd67
15 / 47 pair: 0x600b588d7e847665b2f36cdc70838d04ef9790c2
16 / 47 pair: 0x604229c960e5cacf2aaeac8be68ac07ba9df81c3
17 / 47 pair: 0x609360d5f6dcc26597165ac76e432a16fc27b6fd
18 / 47 pair: 0x634f9d9f8680349

In [10]:
df_old = pd.read_csv("Quickswap export.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x019ba0325f1988213d448b3472fa1cf8d07618d7,2021-07-05 07:09:19.933914,8.630549e+08


In [11]:
df4.to_csv("Quickswap export.csv", index = False)

In [12]:
df_combined = pd.concat([df4.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * 0.003 / df_combined['timedelta']
df_combined.head(3)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x019ba0325f1988213d448b3472fa1cf8d07618d7,2021-07-05 21:20:51.874134,8.643891e+08,2021-07-05 07:09:19.933914,8.630549e+08,0.001619,1.334160e+06,2.472132e+06
0x02e0e14d65175420fb12acd325a7be456bc22005,2021-07-05 21:20:52.088592,2.304381e+05,2021-07-05 07:09:20.141214,2.298131e+05,0.001619,6.249727e+02,1.158043e+03
0x0554059d42e26f35cc958581c71fdfd92405d02f,2021-07-05 21:20:52.299728,4.319752e+05,2021-07-05 07:09:20.334920,4.311199e+05,0.001619,8.552854e+02,1.584801e+03


In [13]:
df4 = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
    date
    dailyVolumeUSD
    reserveUSD
  }
}'''
    response = client('https://api.thegraph.com/subgraphs/name/moonster-bsc/quickswap-daypairdataslim')\
                      .execute(gql(querystring))
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df4 = df4.append(df1)

 1 / 47 pair: 0x019ba0325f1988213d448b3472fa1cf8d07618d7
 2 / 47 pair: 0x02e0e14d65175420fb12acd325a7be456bc22005
 3 / 47 pair: 0x0554059d42e26f35cc958581c71fdfd92405d02f
 4 / 47 pair: 0x0b3a76d33d93b2651dbf03b102eefa9500e89243
 5 / 47 pair: 0x1bd06b96dd42ada85fdd0795f3b4a79db914add5
 6 / 47 pair: 0x1f1e4c845183ef6d50e9609f16f6f9cae43bc9cb
 7 / 47 pair: 0x2aa7a18ceabf2ef893d2f7c0145cc45e6f10b223
 8 / 47 pair: 0x2cf7252e74036d1da831d11089d326296e64a728
 9 / 47 pair: 0x3c986748414a812e455dcd5418246b8fded5c369
10 / 47 pair: 0x4a35582a710e1f4b2030a3f826da20bfb6703c09
11 / 47 pair: 0x4f2cc25028d4d80355c802482463b5b2a93cdb84
12 / 47 pair: 0x527a26525f5483612a1f96cfceacc7032b1c8610
13 / 47 pair: 0x59153f27eefe07e5ece4f9304ebba1da6f53ca88
14 / 47 pair: 0x5ca6ca6c3709e1e6cfe74a50cf6b2b6ba2dadd67
15 / 47 pair: 0x600b588d7e847665b2f36cdc70838d04ef9790c2
16 / 47 pair: 0x604229c960e5cacf2aaeac8be68ac07ba9df81c3
17 / 47 pair: 0x609360d5f6dcc26597165ac76e432a16fc27b6fd
18 / 47 pair: 0x634f9d9f8680349

In [14]:
df5 = df4.merge(df_filtered,
                left_on = 'id',
                right_on = 'id',
                how = 'left',
                validate = 'many_to_one')
df5[['dailyVolumeUSD','reserveUSD']] = df5[['dailyVolumeUSD','reserveUSD']].astype(float)
df5['date'] = pd.to_datetime(df5['date'], unit='s').dt.date
df5['fee'] = round(df5['dailyVolumeUSD'] * 0.003, 6)
df5['APR yday'] = round(100 * df5['fee'] * 365 / df5['reserveUSD'], 3)
df5['Pair'] = '[Quickswap MATIC] ' + df5['Pair']
df5['APR yday Liq + 200'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 200), 3)
df5['APR yday Liq + 2000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 2000), 3)

df5.head(2)

,dailyVolumeUSD,date,reserveUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 2000
0,2.475326e+06,2021-07-05,2.274620e+07,0x019ba0325f1988213d448b3472fa1cf8d07618d7,[Quickswap MATIC] WMATIC-QUICK,WMATIC,QUICK,7425.979451,11.916,11.916,11.915
1,3.693607e+06,2021-07-04,2.291189e+07,0x019ba0325f1988213d448b3472fa1cf8d07618d7,[Quickswap MATIC] WMATIC-QUICK,WMATIC,QUICK,11080.820434,17.652,17.652,17.651


In [15]:
latest_results = df5.sort_values(by = 'date',
                                 ascending = False)\
                    .drop_duplicates(subset = ['Pair'], keep = 'last')\
                    .reset_index(drop = True)\
                    .drop(columns = ['Token 0', 'Token 1'])\
                    .set_index('id')

latest_results.head(2)

,dailyVolumeUSD,date,reserveUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 2000
id,,,,,,,,
0xfba51e1ea2d46c4d5e98f27e1f5333969d2222d7,0.168990,2021-07-04,11.599413,[Quickswap MATIC] LINK-DAI,0.000507,1.595,0.087,0.009
0xab4f7a3199b2cd7a8bfb425941ead08facb6d617,4.045263,2021-07-04,18.600451,[Quickswap MATIC] CRV-WETH,0.012136,23.815,2.026,0.219


In [16]:
df_combined2 = pd.concat([df_combined,
                          df5.sort_values(by = 'date', ascending = False)\
                             .drop_duplicates(subset = ['Pair'],
                                              keep = 'first')\
                             .set_index('id')['reserveUSD']],
                         axis = 1)
df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2 = df_combined2[['APR timedelta', 'APR timedelta Liq + 200', 'APR timedelta Liq + 2000', 'reserveUSD']]
df_combined2.head(2)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 2000,reserveUSD
0x019ba0325f1988213d448b3472fa1cf8d07618d7,10.868332,10.868236,10.867376,2.274620e+07
0x02e0e14d65175420fb12acd325a7be456bc22005,15.457646,15.055717,12.200566,7.491719e+03


In [17]:

df_final_quickswap = pd.concat([df_combined2, latest_results],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 2000',                                            
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 2000',                                            
                                            'reserveUSD',
                                            'dailyVolumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_quickswap.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                              'APR timedelta Liq + 2000',                              
                              'APR yday',
                              'APR yday Liq + 200',
                              'APR yday Liq + 2000',                              
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_quickswap.head(1)

# df_final_quickswap_stylized = df_final_quickswap.dropna()\
#                                                 .style.format({'APR timedelta': "{:0<4,.2f}",
#                                                                'APR timedelta Liq + 100': "{:0<4,.2f}",
#                                                            'APR yday': "{:0<4,.2f}",
#                                                            'APR yday Liq + 100': "{:0<4,.2f}",
#                                                            'Liquidity timedelta': "{:0<4,.0f}",
#                                                            'Liquidity yday': "{:0<4,.0f}",
#                                                            'Volume yday': "{:0<4,.0f}"})

# df_final_quickswap_stylized

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 2000,APR yday,APR yday Liq + 200,APR yday Liq + 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,
[Quickswap MATIC] WMATIC-USDT,2021-07-04,132.42887,132.400867,132.149376,52.166,52.155,52.053,945636.126502,923268.609293,439847.417849


In [18]:
# dfi.export(df_final_quickswap_stylized, 'Quickswap MATIC timedelta.png')
# os.system('xdg-open "Quickswap MATIC timedelta.png"')

## Sushiswap MATIC

In [19]:
# sushiswap_matic = 'https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange'

# sample_transport=RequestsHTTPTransport(url = sushiswap_matic,
#                                        verify = True,
#                                        retries = 3)

# client = Client(transport=sample_transport)

In [20]:
query = gql('''
query {
  factories {
    pairCount
  }
}
''')

response = client('https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange').execute(query)
number_of_pairs = int(response['factories'][0]['pairCount'])
print(number_of_pairs)

1299


In [21]:
try:
    df = pd.read_excel('pairdata_SushiSwap_MATIC.xlsx')
except:
    df = pd.DataFrame([])
    response_dict = {}
    response_list = []


    for i in range(number_of_pairs):
        if i % 100 == 0:
            querystring = '''
                  query {
                  pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                          token0 {symbol}
                          token1 {symbol}
                          id
                      }
                    }
                    '''
            try:
                response = client.execute(gql(querystring))
                df = df.append(pd.DataFrame.from_dict(response['pairs']))
                print(len(df))
            except:
                pass

In [22]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    df.head(2)

In [23]:
criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.to_excel("pairdata_SushiSwap_MATIC.xlsx", index = False)
df_filtered.head()

,id,Pair,Token 0,Token 1
0,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,WMATIC-CRV,WMATIC,CRV
1,0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05,SUSHI-DAI,SUSHI,DAI
2,0x264a05020b491ba1969a325b0a474a864bb0d93b,FRAX-USDT,FRAX,USDT
3,0x2813d43463c374a680f235c428fb1d7f08de0b69,WETH-AAVE,WETH,AAVE
4,0x29bb3ddf61c831adda06439740b340ae48606da6,WBTC-QUICK,WBTC,QUICK


In [24]:
df4 = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client('https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange').execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df4 = df4.append(df1)
    
df4 = df4[['id', 'timestamp', 'volumeUSD']]
df4.head(1)

 1 / 44 pair: 0x140ea3fae80a2732ebc4de0511ff84ef1a575217
 2 / 44 pair: 0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05
 3 / 44 pair: 0x264a05020b491ba1969a325b0a474a864bb0d93b
 4 / 44 pair: 0x2813d43463c374a680f235c428fb1d7f08de0b69
 5 / 44 pair: 0x29bb3ddf61c831adda06439740b340ae48606da6
 6 / 44 pair: 0x34965ba0ac2451a34a0471f04cca3f990b8dea27
 7 / 44 pair: 0x383be588327216586e131e63592a2dc976a16655
 8 / 44 pair: 0x396e655c309676caf0acf4607a868e0cded876db
 9 / 44 pair: 0x3b31bb4b6ba4f67f4ef54e78bcb0aaa4f53dc7ff
10 / 44 pair: 0x3bffd4ca16197ad48dc610c41a022664905b7ffc
11 / 44 pair: 0x3e9a0586088caeb2adff32b0c06cfa44151c7a5b
12 / 44 pair: 0x4b1f1e2435a9c96f7330faea190ef6a7c8d70001
13 / 44 pair: 0x55ff76bffc3cdd9d5fdbbc2ece4528ecce45047e
14 / 44 pair: 0x597a9bc3b24c2a578ccb3aa2c2c62c39427c6a49
15 / 44 pair: 0x5c26e61c6153805b073beb185319de61adbdcb98
16 / 44 pair: 0x5f3eeceead64ee34b78e80cd574496e82e537541
17 / 44 pair: 0x68cce7049013ca8df91cd512cefee8fc0bb8d926
18 / 44 pair: 0x6ff62bfb8c12109

,id,timestamp,volumeUSD
0,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,2021-07-05 21:23:17.084687,996.912484


In [25]:
df_old = pd.read_csv("SushiSwap export.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,2021-07-05 07:11:31.895752,966.8655


In [26]:
df4.to_csv("SushiSwap export.csv", index = False)

In [27]:
df_combined = pd.concat([df4.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * 0.003 / df_combined['timedelta']
df_combined.head(2)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x140ea3fae80a2732ebc4de0511ff84ef1a575217,2021-07-05 21:23:17.084687,996.912484,2021-07-05 07:11:31.895752,966.86550,0.001619,30.046985,55.661153
0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05,2021-07-05 21:23:17.289033,9906.530605,2021-07-05 07:11:32.103242,9885.65665,0.001619,20.873955,38.668389


In [28]:
df4 = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pair:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
    date
    volumeUSD
    reserveUSD
  }
}'''
    response = client('https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange').execute(gql(querystring))
    response
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df4 = df4.append(df1)

 1 / 44 pair: 0x140ea3fae80a2732ebc4de0511ff84ef1a575217
 2 / 44 pair: 0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05
 3 / 44 pair: 0x264a05020b491ba1969a325b0a474a864bb0d93b
 4 / 44 pair: 0x2813d43463c374a680f235c428fb1d7f08de0b69
 5 / 44 pair: 0x29bb3ddf61c831adda06439740b340ae48606da6
 6 / 44 pair: 0x34965ba0ac2451a34a0471f04cca3f990b8dea27
 7 / 44 pair: 0x383be588327216586e131e63592a2dc976a16655
 8 / 44 pair: 0x396e655c309676caf0acf4607a868e0cded876db
 9 / 44 pair: 0x3b31bb4b6ba4f67f4ef54e78bcb0aaa4f53dc7ff
10 / 44 pair: 0x3bffd4ca16197ad48dc610c41a022664905b7ffc
11 / 44 pair: 0x3e9a0586088caeb2adff32b0c06cfa44151c7a5b
12 / 44 pair: 0x4b1f1e2435a9c96f7330faea190ef6a7c8d70001
13 / 44 pair: 0x55ff76bffc3cdd9d5fdbbc2ece4528ecce45047e
14 / 44 pair: 0x597a9bc3b24c2a578ccb3aa2c2c62c39427c6a49
15 / 44 pair: 0x5c26e61c6153805b073beb185319de61adbdcb98
16 / 44 pair: 0x5f3eeceead64ee34b78e80cd574496e82e537541
17 / 44 pair: 0x68cce7049013ca8df91cd512cefee8fc0bb8d926
18 / 44 pair: 0x6ff62bfb8c12109

In [29]:
df5 = df4.merge(df_filtered,
                left_on = 'id',
                right_on = 'id',
                how = 'left',
                validate = 'many_to_one')
df5[['volumeUSD','reserveUSD']] = df5[['volumeUSD','reserveUSD']].astype(float)
df5['date'] = pd.to_datetime(df5['date'], unit='s').dt.date
df5['fee'] = round(df5['volumeUSD'] * 0.003, 6)
df5['APR yday'] = round(100 * df5['fee'] * 365 / df5['reserveUSD'], 3)
df5['Pair'] = '[Sushiswap MATIC] ' + df5['Pair']
df5['APR yday Liq + 200'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 200), 3)
df5['APR yday Liq + 2000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 2000), 3)

df5.head(2)

,date,reserveUSD,volumeUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 2000
0,2021-07-05,580.928508,36.927589,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,[Sushiswap MATIC] WMATIC-CRV,WMATIC,CRV,0.110783,6.961,5.178,1.567
1,2021-07-04,603.253025,52.074449,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,[Sushiswap MATIC] WMATIC-CRV,WMATIC,CRV,0.156223,9.452,7.099,2.190


In [30]:
latest_results = df5.sort_values(by = 'date',
                                 ascending = False)\
                    .drop_duplicates(subset = ['Pair'], keep = 'last')\
                    .reset_index(drop = True)\
                    .drop(columns = ['Token 0', 'Token 1'])\
                    .set_index('id')

latest_results.head(2)

,date,reserveUSD,volumeUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 2000
id,,,,,,,,
0xc4e595acdd7d12fec385e5da5d43160e8a0bac0e,2021-07-04,6.260069e+07,3.143594e+06,[Sushiswap MATIC] WMATIC-WETH,9430.783306,5.499,5.499,5.499
0x5c26e61c6153805b073beb185319de61adbdcb98,2021-07-04,3.196005e+00,1.526368e-01,[Sushiswap MATIC] CRV-USDT,0.000458,5.231,0.082,0.008


In [31]:
df_combined2 = pd.concat([df_combined,
                          df5.sort_values(by = 'date', ascending = False)\
                             .drop_duplicates(subset = ['Pair'],
                                              keep = 'first')\
                             .set_index('id')['reserveUSD']],
                         axis = 1)
df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2 = df_combined2[['APR timedelta', 'APR timedelta Liq + 200', 'APR timedelta Liq + 2000', 'reserveUSD']]
df_combined2.head(2)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 2000,reserveUSD
0x140ea3fae80a2732ebc4de0511ff84ef1a575217,9.581412,7.127561,2.156633,580.928508
0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05,8.250407,5.782755,1.566356,468.684598


In [32]:
df_final_sushiswap_matic = pd.concat([df_combined2, latest_results],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 2000',                                            
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_sushiswap_matic.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                              'APR timedelta Liq + 2000',                              
                              'APR yday',
                              'APR yday Liq + 200',
                              'APR yday Liq + 2000',                              
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_sushiswap_matic.head(1)

# df_final_sushiswap_matic_stylized = df_final_sushiswap_matic.dropna()\
#                                                 .style.format({'APR timedelta': "{:0<4,.2f}",
#                                                                'APR timedelta Liq + 100': "{:0<4,.2f}",
#                                                            'APR yday': "{:0<4,.2f}",
#                                                            'APR yday Liq + 100': "{:0<4,.2f}",
#                                                            'Liquidity timedelta': "{:0<4,.0f}",
#                                                            'Liquidity yday': "{:0<4,.0f}",
#                                                            'Volume yday': "{:0<4,.0f}"})

# df_final_sushiswap_matic_stylized

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 2000,APR yday,APR yday Liq + 200,APR yday Liq + 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,
[Sushiswap MATIC] WMATIC-QUICK,2021-07-04,422.79545,394.147017,244.83675,180.883,168.649,104.835,2751.613155,2757.091377,4554.433328


In [33]:
# dfi.export(df_final_stylized_sushi, 'SushiSwap MATIC timedelta.png')
# os.system('xdg-open "SushiSwap MATIC timedelta.png"')

## Honeyswap Polygon

In [34]:
query = gql('''
query {
  honeyswapFactories {
    pairCount
  }
}
''')

response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(query)
number_of_pairs = int(response['honeyswapFactories'][0]['pairCount'])
print(number_of_pairs)

144


In [35]:
try:
    df = pd.read_excel('pairdata_HoneySwap_MATIC.xlsx')
except:
    df = pd.DataFrame([])
    response_dict = {}
    response_list = []


    for i in range(number_of_pairs):
        if i % 100 == 0:
            querystring = '''
                  query {
                  pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                          token0 {symbol}
                          token1 {symbol}
                          id
                      }
                    }
                    '''
            try:
                response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(gql(querystring))
                df = df.append(pd.DataFrame.from_dict(response['pairs']))
                print(len(df))
            except:
                pass

In [36]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    df.head(2)

In [37]:
criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.to_excel("pairdata_HoneySwap_MATIC.xlsx", index = False)
df_filtered.head(2)

,id,Pair,Token 0,Token 1
0,0x0c787944946d22922c9f41c477cc539700d35bb2,WETH-AAVE,WETH,AAVE
1,0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f,WETH-USDT,WETH,USDT


In [38]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

 1 / 27 pair: 0x0c787944946d22922c9f41c477cc539700d35bb2
 2 / 27 pair: 0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f
 3 / 27 pair: 0x1fc4a2523349bd6df30000b923bb1acb3a27051f
 4 / 27 pair: 0x2177ccb01ec6a3a4d567e42d51f85a65e3ac5ab6
 5 / 27 pair: 0x218e468b15469228f35b0e7f88425cd45fb982bd
 6 / 27 pair: 0x287e9a601a257b24f8d718d08612e9d8ead7f5a8
 7 / 27 pair: 0x35fbfcd7256b3d6bffc69b407e42a135280c0a8b
 8 / 27 pair: 0x3a968d1fe0246de743b6f102e679c11be36688a9
 9 / 27 pair: 0x3f24e142fbf05d16ab1cf79e6df3473f515b16e0
10 / 27 pair: 0x593a7e3c4897834cb6499e051a1ef9532ca82871
11 / 27 pair: 0x5c09be7481c9341a0e4a04843a936429691571a8
12 / 27 pair: 0x6815c440c5c8ad500e1ac6c301f8057b84753268
13 / 27 pair: 0x6d3842ab227a0436a6e8c459e93c74bd8c16fb34
14 / 27 pair: 0x707c48f37a2dacdb729a6880e1c94768d56bbbb0
15 / 27 pair: 0x7fdf6932d8e761c75922af0d4acd04bed36a8c8b
16 / 27 pair: 0x86b7249272fabb82ef36550ef898ea539225e7f0
17 / 27 pair: 0x8df8b187b9a2a7c2d25faa4f33d06c35b25b28bc
18 / 27 pair: 0x96aa3911e89da7b

,id,timestamp,volumeUSD
0,0x0c787944946d22922c9f41c477cc539700d35bb2,2021-07-05 21:23:36.274106,42142.137415


In [39]:
df_old = pd.read_csv("HoneySwap export.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x0c787944946d22922c9f41c477cc539700d35bb2,2021-07-05 07:11:50.224294,36042.79407


In [40]:
df_timedelta.to_csv("HoneySwap export.csv", index = False)

In [41]:
df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * 0.0025 / df_combined['timedelta']
df_combined.head(2)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x0c787944946d22922c9f41c477cc539700d35bb2,2021-07-05 21:23:36.274106,42142.137415,2021-07-05 07:11:50.224294,36042.79407,0.001619,6099.343345,9415.552849
0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f,2021-07-05 21:23:36.459430,113149.229305,2021-07-05 07:11:50.414195,105616.88620,0.001619,7532.343105,11627.674805


In [42]:
df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
    date
    dailyVolumeUSD
    reserveUSD
  }
}'''
    response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(gql(querystring))
    response
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})

 1 / 27 pair: 0x0c787944946d22922c9f41c477cc539700d35bb2
 2 / 27 pair: 0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f
 3 / 27 pair: 0x1fc4a2523349bd6df30000b923bb1acb3a27051f
 4 / 27 pair: 0x2177ccb01ec6a3a4d567e42d51f85a65e3ac5ab6
 5 / 27 pair: 0x218e468b15469228f35b0e7f88425cd45fb982bd
 6 / 27 pair: 0x287e9a601a257b24f8d718d08612e9d8ead7f5a8
 7 / 27 pair: 0x35fbfcd7256b3d6bffc69b407e42a135280c0a8b
 8 / 27 pair: 0x3a968d1fe0246de743b6f102e679c11be36688a9
 9 / 27 pair: 0x3f24e142fbf05d16ab1cf79e6df3473f515b16e0
10 / 27 pair: 0x593a7e3c4897834cb6499e051a1ef9532ca82871
11 / 27 pair: 0x5c09be7481c9341a0e4a04843a936429691571a8
12 / 27 pair: 0x6815c440c5c8ad500e1ac6c301f8057b84753268
13 / 27 pair: 0x6d3842ab227a0436a6e8c459e93c74bd8c16fb34
14 / 27 pair: 0x707c48f37a2dacdb729a6880e1c94768d56bbbb0
15 / 27 pair: 0x7fdf6932d8e761c75922af0d4acd04bed36a8c8b
16 / 27 pair: 0x86b7249272fabb82ef36550ef898ea539225e7f0
17 / 27 pair: 0x8df8b187b9a2a7c2d25faa4f33d06c35b25b28bc
18 / 27 pair: 0x96aa3911e89da7b

In [43]:
df_last_two_days2 = df_last_two_days.merge(df_filtered,
                             left_on = 'id',
                             right_on = 'id',
                             how = 'left',
                             validate = 'many_to_one')
df_last_two_days2[['volumeUSD','reserveUSD']] = df_last_two_days2[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days2['date'] = pd.to_datetime(df_last_two_days2['date'], unit='s').dt.date
df_last_two_days2['fee'] = round(df_last_two_days2['volumeUSD'] * 0.0025, 6)
df_last_two_days2['APR yday'] = round(100 * df_last_two_days2['fee'] * 365 / df_last_two_days2['reserveUSD'], 3)
df_last_two_days2['Pair'] = '[Honeyswap MATIC] ' + df_last_two_days2['Pair']
df_last_two_days2['APR yday Liq + 200'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] + 200), 3)
df_last_two_days2['APR yday Liq + 2000'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] + 2000), 3)

df_last_two_days2.head(2)

,volumeUSD,date,reserveUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 2000
0,7038.035012,2021-07-05,171004.246512,0x0c787944946d22922c9f41c477cc539700d35bb2,[Honeyswap MATIC] WETH-AAVE,WETH,AAVE,17.595088,3.756,3.751,3.712
1,6022.134546,2021-07-04,173625.683842,0x0c787944946d22922c9f41c477cc539700d35bb2,[Honeyswap MATIC] WETH-AAVE,WETH,AAVE,15.055336,3.165,3.161,3.129


In [44]:
df_yesterday = df_last_two_days2.sort_values(by = 'date',
                                             ascending = False)\
                                .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                .reset_index(drop = True)\
                                .drop(columns = ['Token 0', 'Token 1'])\
                                .set_index('id')

df_yesterday.head(2)

,volumeUSD,date,reserveUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 2000
id,,,,,,,,
0x96aa3911e89da7b6d8b4a2b796f3000987d23cf2,1.453006,2021-07-04,900.340356,[Honeyswap MATIC] DAI-USDT,0.003633,0.147,0.121,0.046
0x8df8b187b9a2a7c2d25faa4f33d06c35b25b28bc,0.385519,2021-07-04,19.789163,[Honeyswap MATIC] WMATIC-CRV,0.000964,1.778,0.160,0.017


In [45]:
df_combined2 = pd.concat([df_combined,
                          df_last_two_days2.sort_values(by = 'date', ascending = False)\
                             .drop_duplicates(subset = ['Pair'],
                                              keep = 'first')\
                             .set_index('id')['reserveUSD']],
                         axis = 1)
df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2 = df_combined2[['APR timedelta', 'APR timedelta Liq + 200', 'APR timedelta Liq + 2000', 'reserveUSD']]
df_combined2.head(2)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 2000,reserveUSD
0x0c787944946d22922c9f41c477cc539700d35bb2,5.506035,5.499602,5.442383,171004.246512
0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f,6.235458,6.228778,6.169292,186476.661656


In [46]:
df_final_honeyswap_matic = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 2000',                                            
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_honeyswap_matic.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                              'APR timedelta Liq + 2000',
                              'APR yday',
                              'APR yday Liq + 200',
                              'APR yday Liq + 2000', 
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_honeyswap_matic.head(1)

# df_final_honeyswap_matic_stylized = df_final_honeyswap_matic.dropna()\
#                                                 .style.format({'APR timedelta': "{:0<4,.2f}",
#                                                                'APR timedelta Liq + 100': "{:0<4,.2f}",
#                                                            'APR yday': "{:0<4,.2f}",
#                                                            'APR yday Liq + 100': "{:0<4,.2f}",
#                                                            'Liquidity timedelta': "{:0<4,.0f}",
#                                                            'Liquidity yday': "{:0<4,.0f}",
#                                                            'Volume yday': "{:0<4,.0f}"})

# df_final_honeyswap_matic_stylized

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 2000,APR yday,APR yday Liq + 200,APR yday Liq + 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,
[Honeyswap MATIC] WMATIC-USDC,2021-07-04,19.298283,18.189017,11.98759,6.102,5.985,5.109,3279.467766,10298.549789,688.643063


## Balancer Polygon

In [47]:
# client_exe = client('https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-polygon-v2')
# #https://thegraph.com/explorer/subgraph/balancer-labs/balancer-polygon-v2
# fee = 0.0025

In [48]:
# query = gql('''
#             query {
#                   balancers {
#                     poolCount
#                   }
#                 }
#             ''')

# response = client_exe.execute(query)
# number_of_pairs = int(response['balancers'][0]['poolCount'])
# print(number_of_pairs)

In [49]:
# # make a list of ids and pairs on balancer

# df = pd.DataFrame([])
# response_dict = {}
# response_list = []


# for i in range(number_of_pairs):
#     if i % 100 == 0:
#         querystring = '''
#               query {
#               pools(first: '''+str(100)+' skip: '+str(i)+'''){
#                       tokens{symbol}
#                       id
#                   }
#                 }
#                 '''
#         try:
#             response = client_exe.execute(gql(querystring))
#             df = df.append(pd.DataFrame.from_dict(response['pools']))
#             print(len(df))
#         except:
#             pass
    
# df['tokens'] = df['tokens'].astype(str).str.replace("{'symbol': ",'', regex= True)\
#                                            .replace("}",'', regex= True)\
#                                            .replace("]",'', regex= True)\
#                                            .replace('\[', '', regex = True)\
#                                            .replace("'",'', regex= True)\
#                                            .replace(" ",'', regex= True)

# number_different_tokens = len(df['tokens'].str.split(',', expand = True).columns)
# column_list = []

# for token_column in range(number_different_tokens):
#     column_list.append('Token' + str(token_column + 1))
    
# df[column_list] = df['tokens'].str.split(',', expand = True)
# df.replace()

In [50]:
# # make a list of all tokens on Balancer that are not in my interesting tokens

# df_tokens = pd.DataFrame([])

# for column in column_list:
#     df_tokenpart = df[[column]]
#     df_tokenpart = df_tokenpart.rename(columns = {column:'Token'})
#     df_tokens = df_tokens.append(df_tokenpart)
    
# all_balancer_tokens = df_tokens.drop_duplicates().dropna().sort_values(by = 'Token')['Token'].unique().tolist()
# tokens_to_avoid = set(all_balancer_tokens) - set(interesting_tokens)

In [51]:
# # filter out the pairs on balancer that contain tokens to avoid
# df_filtered = df.copy()

# for column in column_list:
#     df_filtered = df_filtered.loc[~df[column].isin(tokens_to_avoid)]
    
# df_filtered = df_filtered[['id', 'tokens']]
# df_filtered

In [52]:
# df_timedelta = pd.DataFrame([])
# lenid = len(df_filtered['id'])

# for count, id in enumerate(df_filtered['id']):
#     print(f'{count+1:2} / {lenid} pair: {id}')

#     querystring = '''
#                   query {
#       pools(where: {id:"''' + id + '''"}){
#         totalSwapVolume
#       }
#     }'''
#     response = client_exe.execute(gql(querystring))

#     df1 = pd.DataFrame.from_dict(response['pools'])
#     df1['id'] = id
#     df1['timestamp'] = datetime.now()
#     df1['volumeUSD'] = df1['totalSwapVolume'].astype(float)
#     df_timedelta = df_timedelta.append(df1)
    
# df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
# df_timedelta.head(1)

In [53]:
# df_old = pd.read_csv("Balancer export MATIC.csv",
#                      usecols = ['id', 'timestamp', 'volumeUSD'])
# df_old = df_old[['id', 'timestamp', 'volumeUSD']]
# df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
# df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
# df_old.head(1)

In [54]:
# df_timedelta.to_csv("Balancer export MATIC.csv", index = False)

In [55]:
# df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
# df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
# df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
# df_combined['fees per year'] = df_combined['volumeUSD delta'] * fee / df_combined['timedelta']

# df_combined.head(2)

In [56]:
# querystring = """
#               query {
#                     poolSnapshots(orderBy: timestamp, orderDirection: desc){
#                       timestamp
#                     }
#                     }
#                 """

# response = client_exe.execute(gql(querystring))

# df_time = pd.DataFrame.from_dict(response['poolSnapshots'])\
#                       .drop_duplicates()\
#                       .sort_values(by = 'timestamp', ascending = False).head(2)

# df_time['timestamp'].values.tolist()

In [57]:
# {
#   pools{
#     id
#     totalSwapVolume
#     totalLiquidity
#   }
# }

In [58]:
# """
# {
# poolSnapshots(where: {id:"0x36128d5436d2d70cab39c9af9cce146c38554ff0000100000000000000000008-1625270400"} orderBy: timestamp, orderDirection: desc first: 2){
#   amounts
#   totalShares
#   swapVolume
#   swapFees
#   timestamp
# }
# }
# """

In [59]:
# """
# {
# poolSnapshots(where: {id:"0x36128d5436d2d70cab39c9af9cce146c38554ff0000100000000000000000008-1625270400"} orderBy: timestamp, orderDirection: desc first: 2){
#   totalShares
#   swapVolume
#   swapFees
#   timestamp
#   pool{historicalValues(orderBy: block, orderDirection: desc first: 2){
#     poolLiquidity
#     poolId {
#       id
#     }
#     block
#     id
#   }}
# }
# }
# """

In [60]:
# df_last_two_days = pd.DataFrame([])
# lenid = len(df_filtered['id'])

# for count, id in enumerate(df_filtered['id']):
#     print(f'{count+1:2} / {lenid} pair: {id}')
        
#     querystring = '''
#               query {
#   pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
#     date
#     dailyVolumeUSD
#     reserveUSD
#   }
# }'''
#     response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(gql(querystring))
#     response
    
#     df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
#     df1['id'] = id
#     df_last_two_days = df_last_two_days.append(df1)
    
# df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})

## FTM settings

In [61]:
lending_tokens_cream = ['USDC', 'WFTM', 'DAI', 'ETH', 'BTC', 'LINK', 'SUSHI', 'YFI', 'SNX', 'BAND', 'AAVE', 'COVER', 'CREAM', 'HEGIC']
remaining_interesting_tokens = ['WFTM', 'FTM', 'USDC', 'DAI', 'ETH', 'fUSDT', 'FETH', 'WBTC', 'FRAX', 'FRAX', 'ibBTC']

interesting_tokens = list(set(lending_tokens_cream +\
                              remaining_interesting_tokens))

## SpiritSwap

In [62]:
query = gql('''
query {
  spiritswapFactories{
    pairCount
  }
}
''')

response = client('https://api.thegraph.com/subgraphs/name/layer3org/spiritswap-analytics').execute(query)
number_of_pairs = int(response['spiritswapFactories'][0]['pairCount'])
print(number_of_pairs)

710


In [63]:
try:
    df = pd.read_excel('pairdata_Spiritswap_FTM.xlsx')
except:
    df = pd.DataFrame([])
    response_dict = {}
    response_list = []


    for i in range(number_of_pairs):
        if i % 100 == 0:
            querystring = '''
                  query {
                  pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                          token0{symbol}
                          token1{symbol}
                          id
                      }
                    }
                    '''
            try:
                response = client('https://api.thegraph.com/subgraphs/name/layer3org/spiritswap-analytics').execute(gql(querystring))
                df = df.append(pd.DataFrame.from_dict(response['pairs']))
                print(len(df))
            except:
                pass

In [64]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    df.head(2)

In [65]:
criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.to_excel("pairdata_Spiritswap_FTM.xlsx", index = False)
df_filtered.head(2)

,id,Pair,Token 0,Token 1
0,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,WFTM-CREAM,WFTM,CREAM
1,0x0ec0e1629e776272fa3e55548d4a656be0eedcf4,WFTM-FRAX,WFTM,FRAX


In [66]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client('https://api.thegraph.com/subgraphs/name/layer3org/spiritswap-analytics').execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

 1 / 21 pair: 0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a
 2 / 21 pair: 0x0ec0e1629e776272fa3e55548d4a656be0eedcf4
 3 / 21 pair: 0x13bac72a9b0d081aa3926ee7f98d0e1b21dd7d73
 4 / 21 pair: 0x279b2c897737a50405ed2091694f225d83f2d3ba
 5 / 21 pair: 0x33068ad825fbeedb0e57e0466e379290b699f57c
 6 / 21 pair: 0x4556f6afa9b016a83052d459290573a5d83c8764
 7 / 21 pair: 0x4fc38a2735c7da1d71ccabf6dec235a7da4ec52c
 8 / 21 pair: 0x5427e7ab61145c41b1e58453121245daef3929c3
 9 / 21 pair: 0x613bf4e46b4817015c01c6bb31c7ae9edaadc26e
10 / 21 pair: 0x6a65a67da11b68cd141581483a9f7a0cb3c47bae
11 / 21 pair: 0x6d134d63417dc67f57dbc9fcaad4b4cbfab3af2f
12 / 21 pair: 0x6d87359b1e3fd852bd36250018e3328d1afd78e7
13 / 21 pair: 0x9606d683d03f012dda296ef0ae9261207c4a5847
14 / 21 pair: 0x9fe4c0ce5f533e96c2b72d852f190961ad5a7bb3
15 / 21 pair: 0xb67647130d0d51b5a0d53e3ae99ea24ca6100ebb
16 / 21 pair: 0xd061c6586670792331e14a80f3b3bb267189c681
17 / 21 pair: 0xd14dd3c56d9bc306322d4cea0e1c49e9ddf045d4
18 / 21 pair: 0xd3a0c65e5cefcee

,id,timestamp,volumeUSD
0,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,2021-07-05 21:23:49.437320,4.455653e+06


In [67]:
df_old = pd.read_csv("Spiritswap export FTM.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,2021-07-05 07:12:03.420364,4.450440e+06


In [68]:
df_timedelta.to_csv("Spiritswap export FTM.csv", index = False)

In [69]:
df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * 0.003 / df_combined['timedelta']

df_combined.head(2)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,2021-07-05 21:23:49.437320,4.455653e+06,2021-07-05 07:12:03.420364,4.450440e+06,0.001619,5212.318699,9655.510298
0x0ec0e1629e776272fa3e55548d4a656be0eedcf4,2021-07-05 21:23:49.630578,1.061703e+07,2021-07-05 07:12:03.627481,1.059761e+07,0.001619,19426.750957,35986.910167


In [70]:
df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
    date
    dailyVolumeUSD
    reserveUSD
  }
}'''
    response = client('https://api.thegraph.com/subgraphs/name/layer3org/spiritswap-analytics').execute(gql(querystring))
    response
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})
df_last_two_days[['volumeUSD','reserveUSD']] = df_last_two_days[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days['date'] = pd.to_datetime(df_last_two_days['date'], unit='s').dt.date
df_last_two_days.head(1)

 1 / 21 pair: 0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a
 2 / 21 pair: 0x0ec0e1629e776272fa3e55548d4a656be0eedcf4
 3 / 21 pair: 0x13bac72a9b0d081aa3926ee7f98d0e1b21dd7d73
 4 / 21 pair: 0x279b2c897737a50405ed2091694f225d83f2d3ba
 5 / 21 pair: 0x33068ad825fbeedb0e57e0466e379290b699f57c
 6 / 21 pair: 0x4556f6afa9b016a83052d459290573a5d83c8764
 7 / 21 pair: 0x4fc38a2735c7da1d71ccabf6dec235a7da4ec52c
 8 / 21 pair: 0x5427e7ab61145c41b1e58453121245daef3929c3
 9 / 21 pair: 0x613bf4e46b4817015c01c6bb31c7ae9edaadc26e
10 / 21 pair: 0x6a65a67da11b68cd141581483a9f7a0cb3c47bae
11 / 21 pair: 0x6d134d63417dc67f57dbc9fcaad4b4cbfab3af2f
12 / 21 pair: 0x6d87359b1e3fd852bd36250018e3328d1afd78e7
13 / 21 pair: 0x9606d683d03f012dda296ef0ae9261207c4a5847
14 / 21 pair: 0x9fe4c0ce5f533e96c2b72d852f190961ad5a7bb3
15 / 21 pair: 0xb67647130d0d51b5a0d53e3ae99ea24ca6100ebb
16 / 21 pair: 0xd061c6586670792331e14a80f3b3bb267189c681
17 / 21 pair: 0xd14dd3c56d9bc306322d4cea0e1c49e9ddf045d4
18 / 21 pair: 0xd3a0c65e5cefcee

,volumeUSD,date,reserveUSD,id
0,5212.829449,2021-07-05,424838.566108,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a


In [71]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                                                     left_on = 'id',
                                                     right_on = 'id',
                                                     how = 'left',
                                                     validate = 'many_to_one')

df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * 0.003, 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[SpiritSwap FTM] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)

df_last_two_days_incl_pairs.head(1)

,volumeUSD,date,reserveUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 2000
0,5212.829449,2021-07-05,424838.566108,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,[SpiritSwap FTM] WFTM-CREAM,WFTM,CREAM,15.638488,1.344,1.343,1.337


In [72]:
df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

,volumeUSD,date,reserveUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 2000
id,,,,,,,,
0xd061c6586670792331e14a80f3b3bb267189c681,29431.13156,2021-07-04,860713.714899,[SpiritSwap FTM] WFTM-LINK,88.293395,3.744,3.743,3.736


In [73]:
# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2 = df_combined2[['APR timedelta', 'APR timedelta Liq + 200', 'APR timedelta Liq + 2000', 'reserveUSD']]

df_combined2.head(1)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 2000,reserveUSD
0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,2.272748,2.271679,2.262099,424838.566108


In [74]:
df_final_spiritswap = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 2000',                                            
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_spiritswap.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                              'APR timedelta Liq + 2000',
                              'APR yday',
                              'APR yday Liq + 200',
                              'APR yday Liq + 2000', 
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_spiritswap.head(1)

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 2000,APR yday,APR yday Liq + 200,APR yday Liq + 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,
[SpiritSwap FTM] USDC-WFTM,2021-07-04,20.482816,20.481257,20.46723,33.192,33.189,33.168,2.626398e+06,2.729105e+06,827252.647123


## SpookySwap

In [75]:
client_exe = client('https://api.thegraph.com/subgraphs/name/eerieeight/spooky-swap-exchange')
fee = 0.002

In [76]:
query = gql('''
            query {uniswapFactories {pairCount}}
            ''')

response = client_exe.execute(query)
number_of_pairs = int(response['uniswapFactories'][0]['pairCount'])
print(number_of_pairs)

486


In [77]:
# try:
#     df = pd.read_excel('pairdata_Spookyswap_FTM.xlsx')
# except:
df = pd.DataFrame([])
response_dict = {}
response_list = []

for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
          query {
          pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                  token0 {symbol}
                  token1 {symbol}
                  id
              }
            }
            '''
        try:
            response = client_exe.execute(gql(querystring))
            df = df.append(pd.DataFrame.from_dict(response['pairs']))
            print(len(df))
        except:
            pass

100
200
248
248
248


In [78]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    criteria = df['Token 0'].isin(interesting_tokens) & \
               df['Token 1'].isin(interesting_tokens)

    df_filtered = df.loc[criteria,:]
df_filtered.head(1)

,id,Pair,Token 0,Token 1
4,0x06d173628be105fe81f1c82c9979ba79ebcafcb7,WFTM-SNX,WFTM,SNX


In [79]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client_exe.execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

 1 / 21 pair: 0x06d173628be105fe81f1c82c9979ba79ebcafcb7
 2 / 21 pair: 0x2b4c76d0dc16be1c31d4c1dc53bf9b45987fc75c
 3 / 21 pair: 0x484237bc35ca671302d19694c66d617142fbc235
 4 / 21 pair: 0x4a217fbf837c6115b183eaac0245d6a72e8a98e4
 5 / 21 pair: 0x5965e53aa80a0bcf1cd6dbdd72e6a9b2aa047410
 6 / 21 pair: 0x5dc7848bf215f1d99f2af3d2bf78fcdf238ee34b
 7 / 21 pair: 0x7fe139289e3837f8e6d352ecd013c6dbfd903a64
 8 / 21 pair: 0x89d9bc2f2d091cfbfc31e333d6dc555ddbc2fd29
 9 / 21 pair: 0x91b39d5584e2a7dc829f696235742cc293f2e8cf
10 / 21 pair: 0xa572bdf049382f1f98f9a430788dadd51a303969
11 / 21 pair: 0xb77b223490e1f5951ec79a8d09db9eab2adcb934
12 / 21 pair: 0xbf4d61d4cec3a9dff7452d8987e1cc2943e2eb4c
13 / 21 pair: 0xd92206379bd8203ac38225af006bb96bf1f12412
14 / 21 pair: 0xde1a99d2471470c33d9130c5dd485b0c99f3d0df
15 / 21 pair: 0xe120ffbda0d14f3bb6d6053e90e63c572a66a428
16 / 21 pair: 0xebf374bb21d83cf010cc7363918776adf6ff2bf6
17 / 21 pair: 0xec454eda10accdd66209c57af8c12924556f3abd
18 / 21 pair: 0xf0702249f4d3a25

,id,timestamp,volumeUSD
0,0x06d173628be105fe81f1c82c9979ba79ebcafcb7,2021-07-05 21:24:03.214941,4.298927e+06


In [80]:
df_old = pd.read_csv("Spookyswap export.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x06d173628be105fe81f1c82c9979ba79ebcafcb7,2021-07-05 07:12:17.061055,4.101923e+06


In [81]:
df_timedelta.to_csv("Spookyswap export.csv", index = False)

In [82]:
df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * fee / df_combined['timedelta']

df_combined.head(1)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x06d173628be105fe81f1c82c9979ba79ebcafcb7,2021-07-05 21:24:03.214941,4.298927e+06,2021-07-05 07:12:17.061055,4.101923e+06,0.001619,197004.179121,243291.695876


In [84]:
df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
                  query {
                         pairDayDatas(where: {pairAddress:"''' + id + '''"}
                                      orderBy: date
                                      orderDirection: desc 
                                      first: 2) 
                                      {
                            date
                            dailyVolumeUSD
                            reserveUSD}
                        }
                  '''
    response = client_exe.execute(gql(querystring))
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})
df_last_two_days[['volumeUSD','reserveUSD']] = df_last_two_days[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days['date'] = pd.to_datetime(df_last_two_days['date'], unit='s').dt.date
df_last_two_days.head(1)

 1 / 21 pair: 0x06d173628be105fe81f1c82c9979ba79ebcafcb7
 2 / 21 pair: 0x2b4c76d0dc16be1c31d4c1dc53bf9b45987fc75c
 3 / 21 pair: 0x484237bc35ca671302d19694c66d617142fbc235
 4 / 21 pair: 0x4a217fbf837c6115b183eaac0245d6a72e8a98e4
 5 / 21 pair: 0x5965e53aa80a0bcf1cd6dbdd72e6a9b2aa047410
 6 / 21 pair: 0x5dc7848bf215f1d99f2af3d2bf78fcdf238ee34b
 7 / 21 pair: 0x7fe139289e3837f8e6d352ecd013c6dbfd903a64
 8 / 21 pair: 0x89d9bc2f2d091cfbfc31e333d6dc555ddbc2fd29
 9 / 21 pair: 0x91b39d5584e2a7dc829f696235742cc293f2e8cf
10 / 21 pair: 0xa572bdf049382f1f98f9a430788dadd51a303969
11 / 21 pair: 0xb77b223490e1f5951ec79a8d09db9eab2adcb934
12 / 21 pair: 0xbf4d61d4cec3a9dff7452d8987e1cc2943e2eb4c
13 / 21 pair: 0xd92206379bd8203ac38225af006bb96bf1f12412
14 / 21 pair: 0xde1a99d2471470c33d9130c5dd485b0c99f3d0df
15 / 21 pair: 0xe120ffbda0d14f3bb6d6053e90e63c572a66a428
16 / 21 pair: 0xebf374bb21d83cf010cc7363918776adf6ff2bf6
17 / 21 pair: 0xec454eda10accdd66209c57af8c12924556f3abd
18 / 21 pair: 0xf0702249f4d3a25

,volumeUSD,date,reserveUSD,id
0,227477.344505,2021-07-05,1.255352e+06,0x06d173628be105fe81f1c82c9979ba79ebcafcb7


In [85]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                                                     left_on = 'id',
                                                     right_on = 'id',
                                                     how = 'left',
                                                     validate = 'many_to_one')

df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * fee, 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[SpookySwap FTM] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)

df_last_two_days_incl_pairs.head(1)

,volumeUSD,date,reserveUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 2000
0,227477.344505,2021-07-05,1.255352e+06,0x06d173628be105fe81f1c82c9979ba79ebcafcb7,[SpookySwap FTM] WFTM-SNX,WFTM,SNX,454.954689,13.228,13.226,13.207


In [86]:
df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

,volumeUSD,date,reserveUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 2000
id,,,,,,,,
0xe120ffbda0d14f3bb6d6053e90e63c572a66a428,678635.80849,2021-07-04,3.102232e+06,[SpookySwap FTM] WFTM-DAI,1357.271617,15.969,15.968,15.959


In [87]:
# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2 = df_combined2[['APR timedelta', 'APR timedelta Liq + 200', 'APR timedelta Liq + 2000', 'reserveUSD']]

df_combined2.head(1)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 2000,reserveUSD
0x06d173628be105fe81f1c82c9979ba79ebcafcb7,19.38035,19.377263,19.349523,1.255352e+06


In [88]:
df_final_spookyswap = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 2000',                                            
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_spookyswap.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                              'APR timedelta Liq + 2000',
                              'APR yday',
                              'APR yday Liq + 200',
                              'APR yday Liq + 2000', 
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_spookyswap.head(1)

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 2000,APR yday,APR yday Liq + 200,APR yday Liq + 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,
[SpookySwap FTM] USDC-WFTM,2021-07-04,32.868203,32.866604,32.852221,33.725,33.723,33.708,4.111351e+06,4.124383e+06,1.905397e+06


## SushiSwap FTM

In [89]:
client_exe = client('https://api.thegraph.com/subgraphs/name/sushiswap/fantom-exchange')
fee = 0.003

In [90]:
query = gql('''
            query {factories {pairCount}}
            ''')

response = client_exe.execute(query)
number_of_pairs = int(response['factories'][0]['pairCount'])
print(number_of_pairs)

707


In [91]:
# try:
#     df = pd.read_excel('pairdata_Sushiswap_FTM.xlsx')
# except:
df = pd.DataFrame([])
response_dict = {}
response_list = []


for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
              query {
              pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                      token0 {symbol}
                      token1 {symbol}
                      id
                  }
                }
                '''
        try:
            response = client_exe.execute(gql(querystring))
            df = df.append(pd.DataFrame.from_dict(response['pairs']))
            print(len(df))
        except:
            pass
            
df.head(1)

100
200
300
400
421
421
421
421


,id,token0,token1
0,0x0103715fd20a3f2e11fd7b3e646a5f6f6703d245,{'symbol': 'FETH'},{'symbol': 'FUSD'}


In [92]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    criteria = df['Token 0'].isin(interesting_tokens) & \
               df['Token 1'].isin(interesting_tokens)

    df_filtered = df.loc[criteria,:]
df_filtered.head(1)

,id,Pair,Token 0,Token 1
39,0x147e27be8e66f74c7659d88e2e6a13188873d58b,WFTM-CREAM,WFTM,CREAM


In [93]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client_exe.execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

 1 / 27 pair: 0x147e27be8e66f74c7659d88e2e6a13188873d58b
 2 / 27 pair: 0x1ca86e57103564f47ffcea7259a6ce8cc1301549
 3 / 27 pair: 0x27884c7647deb61c9f2b9202d1141bb84661756b
 4 / 27 pair: 0x280abc0d9727f4de3a46ca9cf93c16b4a9c127d1
 5 / 27 pair: 0x2d89bb3b1448152002453aa37fbe4051afdbf56d
 6 / 27 pair: 0x33c89f1f62e4847b6e8595904185594b324c4ba4
 7 / 27 pair: 0x33e29a9ebdd370a8d50656e822abfd3a910da1b6
 8 / 27 pair: 0x3d0bd54c48c2c433ea6fed609cc3d5fb7a77622b
 9 / 27 pair: 0x49d2e0dc99c7358d7a9a8633bf6df111d0ee7f65
10 / 27 pair: 0x52fdb10013bbe9dffe8f27f709a867cb9cc53c60
11 / 27 pair: 0x6aafe06996ac5b5d504ce7ba90dfdd5393679f0d
12 / 27 pair: 0x71c8bceece3daf9e27741d2cc1f03170f862555f
13 / 27 pair: 0x8be92f3d64e91d08ab1cc8a5c487da3f1695b11e
14 / 27 pair: 0x925e0fb729af31d6d387ca188297211a21f92dec
15 / 27 pair: 0xa48869049e36f8bfe0cc5cf655632626988c0140
16 / 27 pair: 0xafa1bf9b2a89dfd60499838c331538283b3346ed
17 / 27 pair: 0xbd0eec56da621b6b31e3d06614f7853624e1c0af
18 / 27 pair: 0xbeddbae6e24314a

,id,timestamp,volumeUSD
0,0x147e27be8e66f74c7659d88e2e6a13188873d58b,2021-07-05 21:33:09.725378,4.017855e+06


In [94]:
df_old = pd.read_csv("Sushiswap export FTM.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x147e27be8e66f74c7659d88e2e6a13188873d58b,2021-07-05 07:12:30.264556,4.014590e+06


In [95]:
df_timedelta.to_csv("Sushiswap export FTM.csv", index = False)

In [96]:
df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * fee / df_combined['timedelta']

df_combined.head(1)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x147e27be8e66f74c7659d88e2e6a13188873d58b,2021-07-05 21:33:09.725378,4.017855e+06,2021-07-05 07:12:30.264556,4.014590e+06,0.001636,3265.781106,5987.171203


In [97]:
df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
                  query {
                         pairDayDatas(where: {pair:"''' + id + '''"}
                                      orderBy: date
                                      orderDirection: desc 
                                      first: 2) 
                                      {
                            date
                            volumeUSD
                            reserveUSD}
                        }
                  '''
    response = client_exe.execute(gql(querystring))
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days[['volumeUSD','reserveUSD']] = df_last_two_days[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days['date'] = pd.to_datetime(df_last_two_days['date'], unit='s').dt.date
df_last_two_days.head(1)

 1 / 27 pair: 0x147e27be8e66f74c7659d88e2e6a13188873d58b
 2 / 27 pair: 0x1ca86e57103564f47ffcea7259a6ce8cc1301549
 3 / 27 pair: 0x27884c7647deb61c9f2b9202d1141bb84661756b
 4 / 27 pair: 0x280abc0d9727f4de3a46ca9cf93c16b4a9c127d1
 5 / 27 pair: 0x2d89bb3b1448152002453aa37fbe4051afdbf56d
 6 / 27 pair: 0x33c89f1f62e4847b6e8595904185594b324c4ba4
 7 / 27 pair: 0x33e29a9ebdd370a8d50656e822abfd3a910da1b6
 8 / 27 pair: 0x3d0bd54c48c2c433ea6fed609cc3d5fb7a77622b
 9 / 27 pair: 0x49d2e0dc99c7358d7a9a8633bf6df111d0ee7f65
10 / 27 pair: 0x52fdb10013bbe9dffe8f27f709a867cb9cc53c60
11 / 27 pair: 0x6aafe06996ac5b5d504ce7ba90dfdd5393679f0d
12 / 27 pair: 0x71c8bceece3daf9e27741d2cc1f03170f862555f
13 / 27 pair: 0x8be92f3d64e91d08ab1cc8a5c487da3f1695b11e
14 / 27 pair: 0x925e0fb729af31d6d387ca188297211a21f92dec
15 / 27 pair: 0xa48869049e36f8bfe0cc5cf655632626988c0140
16 / 27 pair: 0xafa1bf9b2a89dfd60499838c331538283b3346ed
17 / 27 pair: 0xbd0eec56da621b6b31e3d06614f7853624e1c0af
18 / 27 pair: 0xbeddbae6e24314a

,date,reserveUSD,volumeUSD,id
0,2021-07-05,378237.991818,3292.08702,0x147e27be8e66f74c7659d88e2e6a13188873d58b


In [98]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                                                     left_on = 'id',
                                                     right_on = 'id',
                                                     how = 'left',
                                                     validate = 'many_to_one')

df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * fee, 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[SushiSwap FTM] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)

df_last_two_days_incl_pairs.head(1)

,date,reserveUSD,volumeUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 2000
0,2021-07-05,378237.991818,3292.08702,0x147e27be8e66f74c7659d88e2e6a13188873d58b,[SushiSwap FTM] WFTM-CREAM,WFTM,CREAM,9.876261,0.953,0.953,0.948


In [99]:
df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

,date,reserveUSD,volumeUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 2000
id,,,,,,,,
0xbd0eec56da621b6b31e3d06614f7853624e1c0af,2021-07-04,312.74094,0.022102,[SushiSwap FTM] USDC-DAI,0.000066,0.008,0.005,0.001


In [100]:
# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2 = df_combined2[['APR timedelta', 'APR timedelta Liq + 200', 'APR timedelta Liq + 2000', 'reserveUSD']]

df_combined2.head(1)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 2000,reserveUSD
0x147e27be8e66f74c7659d88e2e6a13188873d58b,1.582911,1.582075,1.574585,378237.991818


In [101]:
df_final_sushiswap_ftm = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 2000',                                            
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_sushiswap_ftm.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                              'APR timedelta Liq + 2000',
                              'APR yday',
                              'APR yday Liq + 200',
                              'APR yday Liq + 2000', 
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_sushiswap_ftm.head(1)

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 2000,APR yday,APR yday Liq + 200,APR yday Liq + 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,
[SushiSwap FTM] USDC-WFTM,2021-07-04,68.364036,68.290035,67.631163,68.71,68.635,67.966,184564.37968,182800.945419,114705.640252


In [104]:
df_final_combined = df_final_sushiswap_matic.append(df_final_quickswap)\
                                            .append(df_final_honeyswap_matic)\
                                            .append(df_final_spiritswap)\
                                            .append(df_final_spookyswap)\
                                            .append(df_final_sushiswap_ftm)\
                                            .sort_values(by = 'APR timedelta Liq + 2000',
                                                         ascending = False)

df_final_combined[['APR timedelta',
                   'APR timedelta Liq + 200',
                   'APR timedelta Liq + 2000',                   
                   'APR yday',
                   'APR yday Liq + 200',
                   'APR yday Liq + 2000']] = df_final_combined[['APR timedelta',
                                                                'APR timedelta Liq + 200',
                                                                'APR timedelta Liq + 2000',                   
                                                                'APR yday',
                                                                'APR yday Liq + 200',
                                                                'APR yday Liq + 2000']].round(2)

df_final_combined[['Liquidity timedelta',
                   'Liquidity yday',
                   'Volume yday']] = df_final_combined[['Liquidity timedelta',
                                                        'Liquidity yday',
                                                        'Volume yday']].fillna(0).astype(int)

# criteria = 

df_final_combined.dropna(subset = ['date'])

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 2000,APR yday,APR yday Liq + 200,APR yday Liq + 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,
[Sushiswap MATIC] WMATIC-QUICK,2021-07-04,422.80,394.15,244.84,180.88,168.65,104.84,2751,2757,4554
[Quickswap MATIC] WMATIC-USDT,2021-07-04,132.43,132.40,132.15,52.17,52.16,52.05,945636,923268,439847
[SushiSwap FTM] USDC-WFTM,2021-07-04,68.36,68.29,67.63,68.71,68.64,67.97,184564,182800,114705
[Sushiswap MATIC] SUSHI-WMATIC,2021-07-04,55.03,54.98,54.61,51.65,51.61,51.28,265972,273359,128941
[Sushiswap MATIC] WMATIC-WBTC,2021-07-04,43.15,42.71,39.13,48.73,48.26,44.34,19482,20192,8986
[Sushiswap MATIC] WMATIC-LINK,2021-07-04,68.88,62.53,34.19,94.50,84.70,43.82,1971,1729,1492
[Sushiswap MATIC] WMATIC-USDT,2021-07-04,45.60,44.04,33.70,48.32,46.78,36.35,5663,6077,2681
[Quickswap MATIC] QUICK-USDT,2021-07-04,33.93,33.90,33.60,31.70,31.67,31.40,207562,205407,59471
[SpookySwap FTM] USDC-WFTM,2021-07-04,32.87,32.87,32.85,33.72,33.72,33.71,4111350,4124383,1905397
